In [4]:
import os
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
import scipy.constants as consts
import scipy.integrate as intg
import IPython.display as disp
import matplotlib.pyplot as plt
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Utility Functions

In [16]:
def integrate(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    zsamps = layer_obj.iend - layer_obj.istart
    z_vals = np.linspace(0, layer_obj.thickness, zsamps)
    x_vals = np.linspace(0, sim_proc.period, sim_proc.x_samples)
    y_vals = np.linspace(0, sim_proc.period, sim_proc.y_samples)
    z_integral = intg.trapz(arr_slice, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
    y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
    return y_integral

def compute_fluxes(sim):
    fluxes = sim.flux_dict
    total_incident_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][0])
    total_reflected_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][1])
    total_transmitted_power = .5*sim.period**2/Zo*np.absolute(sum(fluxes['Substrate_bottom']))
    total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
    print('Total Incident Power = {}'.format(total_incident_power))
    print('Total Reflected Power = {}'.format(total_reflected_power))
    print('Total Transmitted Power = {}'.format(total_transmitted_power))
    print('Total Absorbed Power = {}'.format(total_absorbed_power))
    summed_absorbed_power = 0
    abs_dict_fluxmethod = {}
    for layer, (forw_top, back_top) in fluxes.items():
        if '_bottom' in layer:
            continue
        bottom = layer+'_bottom'
        forw_bot, back_bot = fluxes[bottom] 
        print('-'*25)
        print('Layer: {}'.format(layer))
        print('Forward Top: {}'.format(forw_top))
        print('Backward Top: {}'.format(back_top))
        print('Forward Bottom: {}'.format(forw_bot))
        print('Backward Bottom: {}'.format(back_bot))
        P_in = forw_top + -1*back_bot
        P_out = forw_bot + -1*back_top
        print('Power Entering Layer: {}'.format(P_in))
        print('Power Leaving Layer: {}'.format(P_out))
        P_lost = P_in - P_out
        P_abs = .5*P_lost*(sim.period**2)/Zo
        abs_dict_fluxmethod[layer] = P_abs 
        print('Absorbed in Layer: {}'.format(P_abs))
        summed_absorbed_power += P_abs
    print('-'*25)
    print('Summed Absorption= {}'.format(summed_absorbed_power))
    return abs_dict_fluxmethod

# NW w/ Shell

In [5]:
conf = Config('AbsorptionTest.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [15]:
sim.get_fluxes()

{'Air': ((2+0j), (-0.15557938319299336+0j)),
 'Air_bottom': ((2+0j), (-0.15557938319299333+0j)),
 'ITO': ((1.9117439475693845+0.0008713251712570702j),
  (-0.06732333076244565-0.0008713251712570702j)),
 'ITO_bottom': ((1.790140415315491-0.0023688997495998204j),
  (-0.06949267936039395+0.0023688997495998204j)),
 'NW_AlShell': ((1.7313777694253047-0.002189260134698494j),
  (-0.01073003347022824+0.002189260134698494j)),
 'NW_AlShell_bottom': ((0.6199840123149215-0.0035553800448679743j),
  (-0.03289057458249218+0.0035553800448679743j)),
 'NW_SiO2': ((0.6415681690543382-0.0026834809648784153j),
  (-0.0544747313219229+0.0026834809648784153j)),
 'NW_SiO2_bottom': ((0.6326886245562204-0.000383554683423026j),
  (-0.05485000039463061+0.000383554683423026j)),
 'Substrate': ((0.5778386241615838+0j), 0j),
 'Substrate_bottom': ((0.08933583976592889+0j), 0j)}

In [18]:
abs_dict_fluxmethod = compute_fluxes(sim)

Total Incident Power = 0.00016590117058987953
Total Reflected Power = 1.2905400895684514e-05
Total Transmitted Power = 7.410460196398756e-06
Total Absorbed Power = 0.00014558530949779623
-------------------------
Layer: ITO
Forward Top: (1.9117439475693845+0.0008713251712570702j)
Backward Top: (-0.06732333076244565-0.0008713251712570702j)
Forward Bottom: (1.790140415315491-0.0023688997495998204j)
Backward Bottom: (-0.06949267936039395+0.0023688997495998204j)
Power Entering Layer: (1.9812366269297785-0.0014975745783427502j)
Power Leaving Layer: (1.8574637460779366-0.0014975745783427502j)
Absorbed in Layer: (1.0267032910301127e-05+0j)
-------------------------
Layer: Substrate
Forward Top: (0.5778386241615838+0j)
Backward Top: 0j
Forward Bottom: (0.08933583976592889+0j)
Backward Bottom: 0j
Power Entering Layer: (0.5778386241615838+0j)
Power Leaving Layer: (0.08933583976592889+0j)
Absorbed in Layer: (4.0521591883827344e-05+0j)
-------------------------
Layer: Air
Forward Top: (2+0j)
Backw

In [10]:
sim.get_field()

In [12]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: NW_SiO2


Layer: Substrate


In [13]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'ITO': (1.0267032910301127e-05+0j), 'NW_SiO2': (7.676922024955243e-07+0j), 'Substrate': (4.0521591883827344e-05+0j), 'NW_AlShell': (9.402899250116327e-05+0j), 'Air': 0j}
{'Air': 0.0, 'NW_SiO2': 8.5188168329247994e-07, 'Substrate': 3.9391071407509195e-05, 'NW_AlShell': 9.8396810890534838e-05, 'ITO': 1.0272030868605414e-05}


In [14]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

-------------------------
Layer: ITO
Flux Method: (1.0267032910301127e-05+0j)
Integral Method: 1.0272030868605414e-05
Diff: (-4.997958304286802e-09+0j)
Percent Diff: (0.04867967550072082+0j)
-------------------------
Layer: NW_SiO2
Flux Method: (7.676922024955243e-07+0j)
Integral Method: 8.518816832924799e-07
Diff: (-8.418948079695565e-08+0j)
Percent Diff: (10.966567137621341+0j)
-------------------------
Layer: Substrate
Flux Method: (4.0521591883827344e-05+0j)
Integral Method: 3.9391071407509195e-05
Diff: (1.1305204763181496e-06+0j)
Percent Diff: (2.789921184634787+0j)
-------------------------
Layer: NW_AlShell
Flux Method: (9.402899250116327e-05+0j)
Integral Method: 9.839681089053484e-05
Diff: (-4.367818389371564e-06+0j)
Percent Diff: (4.645182590164973+0j)
-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None


# NW no Shell

In [20]:
conf = Config('AbsorptionTestnoshell.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

DEBUG:nanowire.optics.simulate:Logger initialized
DEBUG:nanowire.optics.simulate:Building layer: Air
DEBUG:nanowire.optics.simulate:Layer Order 1
DEBUG:nanowire.optics.simulate:Building layer: ITO
DEBUG:nanowire.optics.simulate:Layer Order 2
DEBUG:nanowire.optics.simulate:Building layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Layer Order 3
DEBUG:nanowire.optics.simulate:Building geometry in layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Building object core of type circle at order 2
DEBUG:nanowire.optics.simulate:Building layer: Substrate
DEBUG:nanowire.optics.simulate:Layer Order 5
DEBUG:nanowire.optics.simulate:Physical Frequency = 4.000000E+14
DEBUG:nanowire.optics.simulate:[399723277333333.31, 400256953271028.0]
DEBUG:nanowire.optics.simulate:[2.3155394055977224e-12, 2.3123628270194848e-12]
INFO:nanowire.optics.simulate:Incident Intensity: 1.23490003178
DEBUG:nanowire.optics.simulate:Incident Power: 0.077181251986
DEBUG:nanowire.optics.simulate:Incident Amplitude: 21.569058302

In [21]:
sim.get_fluxes()

DEBUG:nanowire.optics.simulate:Computing fluxes ...
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Air
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Substrate
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: ITO
DEBUG:nanowire.optics.simulate:Finished computing fluxes!


{'Air': ((2+0j), (-0.2748555116981062+0j)),
 'Air_bottom': ((2+0j), (-0.27485551169810607+0j)),
 'ITO': ((1.7506830520601-0.0015174118232290368j),
  (-0.025538563758270117+0.0015174118232290368j)),
 'ITO_bottom': ((1.6356530688545885+0.0016012904022205607j),
  (-0.02889260987802556-0.0016012904022205607j)),
 'NW_AlShell': ((1.6483629493735117+0.0019215582800875472j),
  (-0.041602490396973384-0.0019215582800875472j)),
 'NW_AlShell_bottom': ((0.9128551463575294+0.005153174357075071j),
  (-0.0759666099963969-0.005153174357075071j)),
 'Substrate': ((0.836888536361133+0j), 0j),
 'Substrate_bottom': ((0.1305585936818471+0j), 0j)}

In [22]:
abs_dict_fluxmethod = compute_fluxes(sim)

Total Incident Power = 0.00016590117058987953
Total Reflected Power = 2.279942556689807e-05
Total Transmitted Power = 1.0829911761193442e-05
Total Absorbed Power = 0.00013227183326178801
-------------------------
Layer: ITO
Forward Top: (1.7506830520601-0.0015174118232290368j)
Backward Top: (-0.025538563758270117+0.0015174118232290368j)
Forward Bottom: (1.6356530688545885+0.0016012904022205607j)
Backward Bottom: (-0.02889260987802556-0.0016012904022205607j)
Power Entering Layer: (1.7795756619381256+8.38785789915239e-05j)
Power Leaving Layer: (1.6611916326128586+8.38785789915239e-05j)
Absorbed in Layer: (9.820024522104203e-06+0j)
-------------------------
Layer: Substrate
Forward Top: (0.836888536361133+0j)
Backward Top: 0j
Forward Bottom: (0.1305585936818471+0j)
Backward Bottom: 0j
Power Entering Layer: (0.836888536361133+0j)
Power Leaving Layer: (0.1305585936818471+0j)
Absorbed in Layer: (5.859048215658802e-05+0j)
-------------------------
Layer: NW_AlShell
Forward Top: (1.64836294937

In [23]:
sim.get_field()

DEBUG:nanowire.optics.simulate:Computing fields ...
DEBUG:nanowire.optics.simulate:Computing up to depth of 3.0700000000000003 microns
DEBUG:nanowire.optics.simulate:Finished computing fields!


In [24]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

DEBUG:nanowire.optics.postprocess:Logger initialized
DEBUG:nanowire.optics.postprocess:Adding normEsquared to data dict


Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: Substrate


In [25]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'Air': 0j, 'ITO': (9.820024522104203e-06+0j), 'Substrate': (5.859048215658802e-05+0j), 'NW_AlShell': (6.386132658308848e-05+0j)}
{'Air': 0.0, 'Substrate': 5.6803853430165159e-05, 'NW_AlShell': 7.5725469836016105e-05, 'ITO': 9.6723173680371644e-06}


In [26]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None
-------------------------
Layer: ITO
Flux Method: (9.820024522104203e-06+0j)
Integral Method: 9.672317368037164e-06
Diff: (1.4770715406703905e-07+0j)
Percent Diff: (1.5041424156789054+0j)
-------------------------
Layer: Substrate
Flux Method: (5.859048215658802e-05+0j)
Integral Method: 5.680385343016516e-05
Diff: (1.7866287264228594e-06+0j)
Percent Diff: (3.0493497589726997+0j)
-------------------------
Layer: NW_AlShell
Flux Method: (6.386132658308848e-05+0j)
Integral Method: 7.57254698360161e-05
Diff: (-1.186414325292762e-05+0j)
Percent Diff: (18.577978078002282+0j)
